In [1]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.

This is by no means production code.
"""
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 2) # update this to your liking

headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

**Selenium**

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x561e2b3f3693 <unknown>
#1 0x561e2b1ecb0a <unknown>
#2 0x561e2b211ef2 <unknown>
#3 0x561e2b20d1f4 <unknown>
#4 0x561e2b248953 <unknown>
#5 0x561e2b242743 <unknown>
#6 0x561e2b218533 <unknown>
#7 0x561e2b219715 <unknown>
#8 0x561e2b4437bd <unknown>
#9 0x561e2b446bf9 <unknown>
#10 0x561e2b428f2e <unknown>
#11 0x561e2b4479b3 <unknown>
#12 0x561e2b41ce4f <unknown>
#13 0x561e2b466ea8 <unknown>
#14 0x561e2b467052 <unknown>
#15 0x561e2b48171f <unknown>
#16 0x7f7b2775cb43 <unknown>


In [35]:
# begin code
url_links = []
property_metadata = defaultdict(dict)

# generate list of urls to visit
for page in N_PAGES:
    url = BASE_URL + f"/rent/melbourne-region-vic/?sort=dateupdated-desc&page={page}" # a single page
    bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser") # makes bs object

    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    index_links = bs_object \
        .find(
            "ul",
            {"data-testid": "results"}
        ) \
        .findAll(
            "a",
            href=re.compile(f"{BASE_URL}/*") # complies RE string into RE expression 
        )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])

print(url_links[0:5])

['https://www.domain.com.au/17-15-truemans-road-capel-sound-vic-3940-16068660', 'https://www.domain.com.au/1-23-panorama-street-clayton-vic-3168-16068659', 'https://www.domain.com.au/unit-15-48-balaclava-rd-st-kilda-east-vic-3183-15600845', 'https://www.domain.com.au/70-tathra-road-wyndham-vale-vic-3024-16068650', 'https://www.domain.com.au/83-285-city-road-southbank-vic-3006-16068649']


In [36]:
test_url = ['https://www.domain.com.au/12-clendon-court-toorak-vic-3142-14238687']
# for each url, scrape some basic metadata
for property_url in test_url:
    bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")

    # looks for the header class to get property name
    property_metadata[property_url]['Name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}) \
        .text

    # regex to find the cost in the summary title 
    cost_finder = re.compile(r'[0-9]+.?[0-9]+') # this regex search assumes that the first numeric value is the cost per week 
    # looks for the div containing a summary title for cost
    cost_text = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extracts the cost from the summary title and adds to dictionary. 
    # if there is no cost written in the summary title, it is replaced by 0 
    cost = cost_finder.search(cost_text)
    if cost == None: 
        property_metadata[property_url]['Cost'] = 0
    else:
         property_metadata[property_url]['Cost'] = cost.group()
        
    # extract coordinates from the hyperlink provided
    # finds latitude and longitude from integrated Google Map
    property_metadata[property_url]['Coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]
    
    # extracts # of bedrooms, # of baths and # of parking spots 
    rooms_info = bs_object.find("div", {"data-testid": "property-features"}).findAll("span", {"data-testid": "property-features-text-container"})
    for i in range(0, len(rooms_info)):
        attr_desc = str(rooms_info[i].text).split(' ')

        property_metadata[property_url][attr_desc[1]] = attr_desc[0]

    # extracts property type from the site 
    property_metadata[property_url]['Property_Type'] = bs_object \
        .find("div", {"data-testid": "listing-summary-property-type"}).find("span", {"class" : "css-in3yi3"}).text

    # extracts desciption from the site -> perhaps can use bag of words to find words that link to more expensive rental units 
    property_metadata[property_url]['Desc'] = bs_object.find("div", {"data-testid" : "listing-details__description"}).findAll("p")
    
     

In [37]:
property_metadata

defaultdict(dict,
            {'https://www.domain.com.au/12-clendon-court-toorak-vic-3142-14238687': {'Name': '12 Clendon Court Toorak VIC 3142',
              'Cost': '2,300',
              'Coordinates': [-37.8444355, 145.0176639],
              'Beds': '4',
              'Baths': '2',
              'Parking': '1',
              'Property_Type': 'House',
              'Desc': [<p>PROPERTY ID: 111991 (quote when calling)</p>,
               <p>APPLY HERE: https://rentbetter.com.au/apply/af873e8b (copy and paste link in your browser)</p>,
               <p>Located in one of Toorak's most secluded enclaves, this recently renovated 4 bedroom home, is walking distance to Toorak Station, trams, shops and restaurants.</p>,
               <p>Offering 4 bedrooms all with BIR, main bedroom with ensuite, family bathroom, 2 separate powder rooms</p>]}})

In [38]:
# output to example json in data/raw/
with open('../data/raw/example.json', 'w') as f:
    dump(property_metadata, f)



TypeError: Object of type Tag is not JSON serializable